In [2]:
from census import Census
from dotenv import load_dotenv
from bson import ObjectId
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='geospatial-data-project')
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap
from folium.vector_layers import Circle, Polygon
import pandas as pd

In [4]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

# Feeding companies_usa with census data
## Get zips from USA

In [271]:
empty_to_none = db.companies_usa.update_many({'offices.zip_code':''}, {'$set':{'offices.zip_code':None}})

In [272]:
empty_to_none.raw_result

{'n': 637, 'nModified': 637, 'ok': 1.0, 'updatedExisting': True}

In [3]:
zips = list(db.companies_usa.find({'offices.zip_code':None}, {'_id':1, 'offices.latitude':1, 'offices.longitude':1}))

In [ ]:
for zip in zips:
    latitude = zip['offices']['latitude']
    longitude = zip['offices']['longitude']
    id = ObjectId(zip['_id'])
    try:
        location = geolocator.reverse(f"{latitude},{longitude}")
        db.companies_usa.update_one({'_id':id}, {'$set':{'offices.zip_code': location.raw['address']['postcode']}})
        leftovers_zips_usa.append({'_id':id, 'loc_params':location.raw})
    except:
        location = geolocator.reverse(f"{latitude},{longitude}")
        db.companies_usa.update_one({'_id':id}, {'$set':{'offices.more_location': location.raw}})
        no_post_code.append({'_id':id, 'coord':{'latitude':latitude, 'longitude':longitude}})

## Add demographics to companies_usa

In [30]:
load_dotenv()
census_api_k = os.getenv('census_api_key')
c = Census(census_api_k, year=2018)

In [57]:
def demographics_age(finded):
    counter = 0
    for comp in finded:
        id = ObjectId(comp['_id'])
        zip_code = comp['offices']['zip_code']
        try:
            demographics = {'age': 
                                {'median_age': c.acs5dp.zipcode('DP05_0018E', zip_code)[0]['DP05_0018E'], 
                                'range_25_34': c.acs5dp.zipcode('DP05_0010PE', zip_code)[0]['DP05_0010PE'], 
                                'range_35_44': c.acs5dp.zipcode('DP05_0011PE', zip_code)[0]['DP05_0011PE']}
                            }
            db.companies_usa.update_one({'_id':id}, {'$set':{'offices.demographics': demographics}})
        except:
            counter += 1
    return f'Failed to append demographics in {counter} documents'

In [ ]:
usa_all = list(db.companies_usa.find({'offices.zip_code':{'$ne':None}, 'offices.demographics':{'$exists':False}}, {'_id':1, 'offices.zip_code':1}))
demographics_age(usa_all)

### Some zips are longer than expected

In [49]:
long_zips = list(db.companies_usa.find({'offices.demographics':{'$exists':False}, 'offices.zip_code':{'$ne':None}}, {'_id':1, 'offices.zip_code':1}))
len(long_zips)

525

In [54]:
for lzip in long_zips:
    lzip['offices']['zip_code'] = lzip['offices']['zip_code'][:5]

In [58]:
demographics_age(long_zips)

'Failed to append demographics in 255 documents'

In [63]:
have_demo = len(list(db.companies_usa.find({'offices.demographics':{'$exists':True}})))
no_demo = len(list(db.companies_usa.find({'offices.demographics':{'$exists':False}})))
print(f"There are {have_demo} documents with demographic data and {no_demo} without")

There are 7304 documents with demographic data and 296 without


## Filter and new collection

In [71]:
age_filtered = list(db.companies_usa.find({'offices.demographics.age.median_age':{'$lte':40}, 
                                           '$where': '[this.offices.demographics.age.range_25_34 + this.offices.demographics.age.range_35_44] >= 45'}))

In [72]:
len(age_filtered)

1390

In [74]:
db.companies_usa_filt1.insert_many(age_filtered)

# Map

In [5]:
offices_no_filter = list(db.companies_usa.find({'offices.demographics':{'$exists':True}}, {'offices':1}))

to_df = {'_id': [], 'latitude': [], 'longitude': [], 'median_age': [], 'range_25_44': []}

for office in offices_no_filter:
    to_df['_id'].append(office['_id'])
    to_df['latitude'].append(office['offices']['latitude'])
    to_df['longitude'].append(office['offices']['longitude'])
    to_df['median_age'].append(office['offices']['demographics']['age']['median_age'])
    to_df['range_25_44'].append(office['offices']['demographics']['age']['range_25_34'] + office['offices']['demographics']['age']['range_35_44'])

In [6]:
ages_df = pd.DataFrame.from_dict(to_df)
ages_df.head()

indexes = ages_df[(ages_df['median_age'] < 0) | (ages_df['range_25_44'] < 0)].index
ages_df.drop(indexes, inplace=True)

In [7]:
m_heat = Map(location=[37.0902,-95.7129],zoom_start=4)

offices = HeatMap(data=ages_df[["latitude","longitude"]], name='Offices', radius=10)
fg = FeatureGroup(name="Offices")
offices.add_to(fg)
fg.add_to(m_heat)

median_age = HeatMap(data=ages_df[["latitude","longitude", 'median_age']], name='Median Age', max_val=50, radius=10)
fg_m_age = FeatureGroup(name="Median Age")
median_age.add_to(fg_m_age)
fg_m_age.add_to(m_heat)

range_25_44 = HeatMap(data=ages_df[["latitude","longitude", 'range_25_44']], name='People 25/44 YO', max_val=ages_df['range_25_44'].max(), radius=10)
fg_range = FeatureGroup(name='People 25/44 YO')
range_25_44.add_to(fg_range)
fg_range.add_to(m_heat)

filter_1 = ages_df[(ages_df['median_age'] <= 40) & (ages_df['range_25_44'] >= 45)]
filter_pass = HeatMap(data=filter_1[["latitude","longitude"]], name='Offices selected', radius=10)
f_pass = FeatureGroup(name='Offices selected')
filter_pass.add_to(f_pass)
f_pass.add_to(m_heat)

LayerControl(collapsed=False).add_to(m_heat)
m_heat